In [20]:
import torch
from chadhmm.hmm import MultinomialHMM, GaussianHMM, GaussianMixtureHMM, PoissonHMM
from chadhmm.utilities import constraints

In [21]:
hmm = MultinomialHMM(n_states=6,
                     n_features=4,
                     n_trials=2,
                     transitions=constraints.Transitions.ERGODIC)
train_seq = torch.randint(0,hmm.n_features,(1000,))
one_hot = hmm.n_trials * torch.nn.functional.one_hot(train_seq,4)

In [22]:
print(f'pi {hmm._params.pi.exp()}\n' +
      f'A  {hmm._params.A.exp()}\n' +
      f'B  {hmm._params.B.exp()}')

pi tensor([0.0039, 0.2151, 0.1617, 0.3731, 0.0109, 0.2353], dtype=torch.float64)
A  tensor([[0.2015, 0.2020, 0.1287, 0.0593, 0.0109, 0.3975],
        [0.2456, 0.3214, 0.3009, 0.0333, 0.0041, 0.0945],
        [0.0557, 0.0696, 0.2570, 0.1695, 0.3092, 0.1390],
        [0.6473, 0.0181, 0.1158, 0.0855, 0.0595, 0.0737],
        [0.0370, 0.0608, 0.3788, 0.4453, 0.0480, 0.0301],
        [0.0416, 0.1121, 0.1779, 0.2990, 0.1687, 0.2007]], dtype=torch.float64)
B  tensor([[0.0127, 0.7084, 0.2612, 0.0177],
        [0.1421, 0.5096, 0.3021, 0.0462],
        [0.1634, 0.4938, 0.0959, 0.2469],
        [0.0444, 0.6202, 0.2446, 0.0908],
        [0.0937, 0.5572, 0.2470, 0.1021],
        [0.2430, 0.1831, 0.0555, 0.5184]], dtype=torch.float64)


In [18]:
hmm.fit(X=one_hot,max_iter=1,n_init=1,verbose=False)

MultinomialHMM(
  (_params): ParameterDict(
      (B): Parameter containing: [torch.DoubleTensor of size 6x4]
      (A): Parameter containing: [torch.DoubleTensor of size 6x6]
      (pi): Parameter containing: [torch.DoubleTensor of size 6]
  )
)

In [6]:
obs = hmm.to_observations(one_hot,lengths=[400,600])
gamma_vec,_ = hmm._compute_posteriors(obs)

In [8]:
torch.cat(gamma_vec).exp()

tensor([[6.0768e-01, 7.5461e-02, 1.7553e-01, 1.3717e-01, 3.9977e-03, 1.5940e-04],
        [2.2572e-03, 1.2661e-01, 1.4318e-01, 4.3402e-01, 2.9118e-01, 2.7405e-03],
        [1.2086e-02, 3.8191e-01, 3.9938e-03, 4.0441e-01, 4.3907e-02, 1.5370e-01],
        ...,
        [1.0923e-03, 1.0919e-01, 6.8193e-02, 2.6563e-01, 5.5402e-01, 1.8760e-03],
        [3.0470e-03, 5.2875e-02, 1.8025e-01, 1.6453e-01, 5.9787e-01, 1.4275e-03],
        [1.4091e-02, 2.3405e-01, 3.5096e-03, 5.0780e-01, 3.7215e-02, 2.0334e-01]],
       dtype=torch.float64)

In [3]:
inf = hmm.ic(
    one_hot,
    lengths=[600,400],
    criterion=constraints.InformCriteria.BIC
)
inf

tensor([3350.9500, 2428.6436], dtype=torch.float64)

In [4]:
hmm.score(one_hot,lengths=[600,400])

tensor([-1492.4195, -1031.2663], dtype=torch.float64)

In [5]:
hmm.predict(one_hot,lengths=[500,500])

(tensor([-1243.5297, -1280.5941], dtype=torch.float64),
 [tensor([5, 4, 1, 1, 0, 4, 1, 1, 0, 4, 1, 3, 3, 4, 5, 4, 4, 4, 5, 3, 3, 3, 4, 0,
          4, 0, 3, 3, 4, 0, 4, 5, 4, 5, 4, 5, 4, 0, 5, 4, 4, 1, 3, 3, 4, 4, 4, 1,
          3, 3, 3, 5, 4, 5, 3, 5, 4, 0, 3, 2, 1, 1, 3, 3, 2, 5, 2, 5, 4, 1, 3, 4,
          1, 0, 2, 1, 3, 4, 4, 5, 2, 5, 1, 3, 3, 2, 5, 1, 1, 1, 3, 3, 3, 5, 4, 0,
          4, 0, 3, 3, 4, 0, 4, 4, 5, 3, 3, 4, 1, 1, 0, 2, 4, 1, 1, 1, 3, 3, 3, 2,
          5, 4, 4, 0, 2, 5, 2, 5, 2, 1, 3, 3, 4, 0, 4, 5, 2, 1, 3, 3, 4, 4, 2, 5,
          4, 0, 4, 0, 5, 4, 0, 4, 0, 4, 0, 2, 1, 1, 1, 3, 3, 3, 4, 4, 0, 4, 0, 3,
          3, 2, 5, 4, 0, 2, 4, 4, 1, 1, 3, 3, 3, 4, 0, 4, 1, 1, 1, 1, 3, 3, 4, 4,
          0, 4, 1, 3, 2, 1, 3, 3, 3, 3, 4, 1, 3, 2, 5, 2, 1, 0, 4, 4, 4, 0, 5, 4,
          5, 3, 2, 5, 1, 3, 2, 5, 3, 4, 0, 3, 4, 5, 3, 3, 4, 0, 3, 3, 3, 3, 5, 4,
          0, 4, 5, 3, 5, 4, 0, 2, 4, 4, 0, 2, 5, 3, 5, 4, 4, 5, 4, 5, 3, 4, 0, 2,
          3, 3, 4, 1, 1, 3, 2, 5, 2, 5, 4,

In [2]:
gmm = GaussianHMM(n_states=2,n_features=2)
train_seq = torch.rand((1000,2), dtype=torch.float64)
gmm.fit(X=train_seq,max_iter=5, n_init=1, verbose=True)

Run 1 | Initialization | Score: -2163.71


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x2 and 1000x2)

In [ ]:
phmm = PoissonHMM(n_states=2,n_features=2)
train_seq = torch.randint(1,100,(100,2))
phmm.fit(X=train_seq,max_iter=15,lengths=[50,50],n_init=1,verbose=False)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x2 and 100x2)

In [3]:
gmmm = GaussianMixtureHMM(n_states=4,
                         n_components=3,
                         n_features=3)
train_seq = torch.rand((1000,3),dtype=torch.float64)
gmmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=True)

Run 1 | Initialization | Score: -3260.42


RuntimeError: The size of tensor a (1000) must match the size of tensor b (4) at non-singleton dimension 2

In [ ]:
import torch
from chadhmm.hsmm import CategoricalHSMM, GaussianHSMM, GaussianMixtureHSMM, PoissonHSMM

In [ ]:
hsmm = CategoricalHSMM(n_states=5,
                       n_features=4,
                       max_duration=4)
train_seq = torch.randint(0, 4, (100,))
hsmm.fit(X=train_seq,max_iter=15, n_init=1)

TypeError: 'int' object is not iterable

In [ ]:
print(f'pi {hsmm.params.pi.exp().sum()}\n'+
      f'A  {hsmm.params.A.exp().sum(1)}\n' +
      f'D  {hsmm.params.D.exp().sum(1)}\n' +
      f'B  {hsmm.params.B.exp().sum(1)}')

In [ ]:
ghsmm = GaussianHSMM(n_states=5,
                  n_features=3,
                  max_duration=4)
train_seq = torch.rand((100,3), dtype=torch.float64)
ghsmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

In [ ]:
gmhsmm = GaussianMixtureHSMM(n_states=4,
                         n_components=3,
                         max_duration=4,
                         n_features=3)
train_seq = torch.rand((100,3),dtype=torch.float64)
gmhsmm.fit(X=train_seq,max_iter=15, n_init=1, verbose=False)

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical, Multinomial

Multinomial(2,probs=torch.Tensor([0.2,0.8]))

In [ ]:
m = Multinomial(1,logits=hmm.pdf.logits)
m.log_prob(torch.Tensor([0,0,0,1])).exp()

In [ ]:
hmm.pdf.log_prob(train_seq[4])